In [1]:
import os
import random
import numpy as np
import torch

SEED = 42

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # For CUDA
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Ensures deterministic behavior for cuDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
from pathlib import Path
import re
import warnings
import pandas as pd
import pickle  # --- IMPORT PICKLE ---
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ema = pd.read_csv("/content/drive/MyDrive/CraveMultimodal/data/EMA/CombineEMA.csv")
ema['User_ID'].unique()

array(['CRS022', 'CRS026', 'CRS020', 'CRS018', 'CRS023', 'CRS021',
       'CRS019', 'CRS024', 'CRS017', 'CRS015', 'CRS014', 'CRS016',
       'CRS012', 'CRS013', 'CRS011', 'CRS010', 'CRS008', 'CRS007',
       'CRS009', 'CRS002', 'CRS001', 'CRS006', 'CRS005', 'CRS003',
       'CR021', 'CR010', 'CR001', 'CR008', 'CR006', 'CR004', 'CR016',
       'CR005', 'CR022', 'CR025', 'CR003', 'CR020', 'CR002', 'CR018',
       'CR027', 'CR015', 'CR009', 'CR011', 'CR012', 'CR031', 'CR032',
       'CR035', 'CR023'], dtype=object)

In [6]:
ema["User_ID"] = ema["User_ID"].str.lower()

In [7]:
df_fitbit_hr = pd.read_csv("/content/drive/MyDrive/CraveMultimodal/data/Combined_Fitbit/Combined_Fitbit_HR.csv")

In [8]:
df_fitbit_hr["date"] = pd.to_datetime(df_fitbit_hr["date"])
df_fitbit_hr['subject']=df_fitbit_hr['subject'].str.lower()
df_fitbit_hr.head()

,subject,date,time,value
0,cr001,2021-07-22,00:00:00,64
1,cr001,2021-07-22,00:01:00,57
2,cr001,2021-07-22,00:02:00,59
3,cr001,2021-07-22,00:03:00,96
4,cr001,2021-07-22,00:04:00,115


In [9]:
def pad_or_trim(x, T):
    if len(x) >= T:
        return x[:T]
    else:
        return np.pad(x, (0, T - len(x)), mode='constant')

In [10]:
all_day_sequences = []
T = 1420   # The fixed length for the time series (1000 minutes = ~16.7 hours)


#  Group the data first by Subject, then by Date
subject_day_groups = df_fitbit_hr.groupby(['subject', 'date'])

#  Iterate through the groups, extract the sequence, and pad/trim it
for (subj, date), group_df in subject_day_groups:
    # 'group_df' is the DataFrame slice for one subject on one date

    # Extract the heart rate values as a NumPy array
    hr_seq = group_df['value'].values

    # Apply the padding/trimming function
    hr_seq_fixed = pad_or_trim(hr_seq, T)

    # Append the fixed-length sequence
    all_day_sequences.append(hr_seq_fixed)

#  Convert the list of sequences into the final 2D NumPy array
all_day_sequences = np.array(all_day_sequences)

print(f"Total number of daily sequences (N_days): {all_day_sequences.shape[0]}")
print(f"Length of each sequence (T): {all_day_sequences.shape[1]}")
print(f"Final array shape: {all_day_sequences.shape}")


Total number of daily sequences (N_days): 759
Length of each sequence (T): 1420
Final array shape: (759, 1420)


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class HR_Autoencoder(nn.Module):
    def __init__(self, seq_len):
        super().__init__()
        self.seq_len = seq_len

        # ---- Encoder ----
        self.encoder_conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),          # T -> T/2

            nn.Conv1d(16, 32, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),          # T/2 -> T/4
        )

        # figure out encoded length
        with torch.no_grad():
            dummy = torch.zeros(1, 1, seq_len)
            enc_out = self.encoder_conv(dummy)
        self.enc_channels = enc_out.shape[1]   # 32
        self.enc_len = enc_out.shape[2]        # T/4

        # compress to latent vector
        self.fc_enc = nn.Linear(self.enc_channels * self.enc_len, 64)  # latent dim = 64
        self.fc_dec = nn.Linear(64, self.enc_channels * self.enc_len)

        #  Decoder
        self.decoder_conv = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="nearest"),  # T/4 -> T/2
            nn.Conv1d(32, 16, kernel_size=5, padding=2),
            nn.ReLU(),

            nn.Upsample(scale_factor=2, mode="nearest"),  # T/2 -> T
            nn.Conv1d(16, 1, kernel_size=5, padding=2),
        )

    def encode(self, x):
        # x: (B, 1, T)
        h = self.encoder_conv(x)                  # (B, C, L_enc)
        h_flat = h.view(h.size(0), -1)            # (B, C * L_enc)
        z = self.fc_enc(h_flat)                   # (B, 32)
        return z

    def decode(self, z):
        # z: (B, 32)
        h_flat = self.fc_dec(z)                   # (B, C * L_enc)
        h = h_flat.view(-1, self.enc_channels, self.enc_len)  # (B, C, L_enc)
        x_recon = self.decoder_conv(h)            # (B, 1, ~T)

        # ensure output length exactly seq_len (crop or pad)
        if x_recon.shape[-1] > self.seq_len:
            x_recon = x_recon[..., :self.seq_len]
        elif x_recon.shape[-1] < self.seq_len:
            pad_len = self.seq_len - x_recon.shape[-1]
            x_recon = F.pad(x_recon, (0, pad_len))
        return x_recon

    def forward(self, x):
        z = self.encode(x)
        x_recon = self.decode(z)
        return x_recon, z


In [12]:
from torch.utils.data import Dataset, DataLoader

class HRDayDataset(Dataset):
    def __init__(self, sequences):
        # sequences: (N_days, T)
        self.sequences = torch.tensor(sequences, dtype=torch.float32)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        x = self.sequences[idx]      # (T,)
        x = x.unsqueeze(0)           # (1, T)  channel dim
        return x, x                  # autoencoder: target = input


In [13]:
def train_autoencoder(model, dataloader, epochs=20, lr=1e-3, device="cpu"):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for ep in range(epochs):
        model.train()
        total_loss = 0.0
        for batch_x, batch_y in dataloader:
            batch_x = batch_x.to(device)  # (B, 1, T)
            batch_y = batch_y.to(device)

            opt.zero_grad()
            x_recon, _ = model(batch_x)
            loss = loss_fn(x_recon, batch_y)
            loss.backward()
            opt.step()

            total_loss += loss.item() * batch_x.size(0)

        avg_loss = total_loss / len(dataloader.dataset)
        print(f"Epoch {ep+1}/{epochs}, Loss: {avg_loss:.6f}")


In [14]:
#Train
T = 1420  # or whatever length you padded/trimmed to
dataset = HRDayDataset(all_day_sequences)   # shape (N_days, T)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = HR_Autoencoder(seq_len=T)
train_autoencoder(model, dataloader, epochs=20, lr=1e-3, device="cuda" if torch.cuda.is_available() else "cpu")

Epoch 1/20, Loss: 1259.433433
Epoch 2/20, Loss: 335.407886
Epoch 3/20, Loss: 252.969108
Epoch 4/20, Loss: 204.329902
Epoch 5/20, Loss: 164.528525
Epoch 6/20, Loss: 136.072326
Epoch 7/20, Loss: 114.343474
Epoch 8/20, Loss: 98.987007
Epoch 9/20, Loss: 89.209131
Epoch 10/20, Loss: 80.998277
Epoch 11/20, Loss: 76.716793
Epoch 12/20, Loss: 73.289837
Epoch 13/20, Loss: 66.105903
Epoch 14/20, Loss: 63.532742
Epoch 15/20, Loss: 59.861984
Epoch 16/20, Loss: 58.624354
Epoch 17/20, Loss: 57.814247
Epoch 18/20, Loss: 56.098222
Epoch 19/20, Loss: 53.572229
Epoch 20/20, Loss: 52.245825


In [15]:
day_embeddings = {}
T = 1420 # Your fixed length

#  PyTorch Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
model.eval()
model.to(device)

#  Group the data by Subject and then by Date
# This efficiently yields the daily sequences you need.
subject_day_groups = df_fitbit_hr.groupby(['subject', 'date'])

# Iterate through the groups
for (subj, date), group_df in subject_day_groups:

    # Initialize the inner dictionary for the subject if it's the first time seeing them
    if subj not in day_embeddings:
        day_embeddings[subj] = {}

    #  Extract the heart rate sequence
    hr_seq = group_df['value'].values

    #  Pad/Trim the sequence to the fixed length T
    hr_seq_fixed = pad_or_trim(hr_seq, T)

    # Convert sequence to a PyTorch tensor
    # .unsqueeze(0).unsqueeze(0) converts (T,) -> (1, 1, T) for a typical 1D CNN/RNN input
    x = torch.tensor(hr_seq_fixed, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

    #  Run the model to get the embedding (z)
    with torch.no_grad():
        # Assuming model(x) returns (output, embedding)
        _, z = model(x)          # z: (1, 32) (batch size 1, embedding dim 32)

    # Store the resulting embedding
    # .cpu() moves to CPU, .numpy() converts to numpy array, .squeeze(0) removes the batch dimension
    day_embeddings[subj][date] = z.cpu().numpy().squeeze(0)  # Stores the embedding as a (32,) array

print(f"Finished generating embeddings for {len(day_embeddings)} subjects.")

Finished generating embeddings for 35 subjects.


In [16]:
column_extract = ["User_ID","Start","End","Positive","Negative"]
df_ema = ema[column_extract]
df_ema["Start"] = pd.to_datetime(df_ema["Start"], format="mixed", errors='coerce')
df_ema["End"] = pd.to_datetime(df_ema["End"], format= "mixed",errors = 'coerce')
df_ema.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3562 entries, 0 to 3561
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   User_ID   3562 non-null   object        
 1   Start     3562 non-null   datetime64[ns]
 2   End       3562 non-null   datetime64[ns]
 3   Positive  3562 non-null   float64       
 4   Negative  3562 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 139.3+ KB


/tmp/ipython-input-3332197160.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ema["Start"] = pd.to_datetime(df_ema["Start"], format="mixed", errors='coerce')
/tmp/ipython-input-3332197160.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ema["End"] = pd.to_datetime(df_ema["End"], format= "mixed",errors = 'coerce')


In [17]:
df_ema["Date_only"] = df_ema["Start"].dt.date
df_ema["Date_only"] = pd.to_datetime(df_ema["Date_only"])
df_ema = df_ema[["User_ID", "Date_only","Positive","Negative"]]


/tmp/ipython-input-2322932470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ema["Date_only"] = df_ema["Start"].dt.date
/tmp/ipython-input-2322932470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ema["Date_only"] = pd.to_datetime(df_ema["Date_only"])


In [18]:
df_ema_new = df_ema.copy()

def ema_multiclass(row):
    p = int(row["Positive"])
    n = int(row["Negative"])
    if p == 0 and n == 0:
        return 0  # neither
    elif p == 1 and n == 0:
        return 1  # positive only
    elif p == 0 and n == 1:
        return 2  # negative only
    elif p == 1 and n == 1:
        return 3  # both
    else:
        return None  # just in case of weird values

df_ema_new["EMA_class"] = df_ema_new.apply(ema_multiclass, axis=1)

# Keep only positive-only (1) and negative-only (2)
mask_pos_only = df_ema_new["EMA_class"] == 1
mask_neg_only = df_ema_new["EMA_class"] == 2

df_ema_bin = df_ema_new[mask_pos_only | mask_neg_only].copy()

# New binary label:
# 1 = positive only, 0 = negative only
df_ema_bin["EMA_bin"] = np.where(df_ema_bin["EMA_class"] == 1, 1, 0)
df_ema_bin.head()



,User_ID,Date_only,Positive,Negative,EMA_class,EMA_bin
0,crs022,2025-07-16,1.0,0.0,1,1
2,crs022,2025-07-22,1.0,0.0,1,1
3,crs022,2025-07-06,1.0,0.0,1,1
4,crs022,2025-07-06,1.0,0.0,1,1
6,crs022,2025-07-08,1.0,0.0,1,1


In [19]:
#Build a (subject, date) class mapping

#ema_labels_multi = {}
ema_labels_bin = {}

for subj, df in df_ema_bin.groupby("User_ID"):    #change
  ema_labels_bin[subj] = {}                      #change
  for _, row in df.iterrows():
    date = pd.to_datetime(row["Date_only"]).date()
    ema_labels_bin[subj][date] = int(row["EMA_bin"]) #change

In [20]:
from datetime import date



new_day_embeddings = {}

#  Iterate through each subject in the outer dictionary
for subj, date_data in day_embeddings.items():

    # k is the Timestamp, v is the NumPy array
    # k.date() extracts the datetime.date object from the Timestamp
    converted_date_data = {
        k.date(): v
        for k, v in date_data.items()
    }

    # Store the result in the new dictionary
    new_day_embeddings[subj] = converted_date_data

# The original variable name
day_embeddings = new_day_embeddings

Binary Prediction using gru with fixed H

In [21]:
EMBEDDING_DIM = 32
ZERO_EMBEDDING = np.zeros(EMBEDDING_DIM, dtype=np.float32)

def build_dataset_multiclass(day_embeddings, ema_labels_bin, H=7): #change
    X_list, y_list, subj_list = [], [], []

    for subj in day_embeddings:
        if subj not in ema_labels_bin: #change
            continue



        days = sorted(day_embeddings[subj].keys())

        for i in range(H, len(days)):
            day_n = days[i]
            prev_days = days[i-H:i]

            # Target Label Check (Uses the fixed dictionary)
            if day_n not in ema_labels_bin[subj]: #change
                continue


            seq = np.vstack([day_embeddings[subj][d]for d in prev_days])

            X_list.append(seq)

            y_list.append(ema_labels_bin[subj][day_n]) #change

            subj_list.append(subj)

    return np.array(X_list), np.array(y_list), np.array(subj_list)

In [22]:
X_mc, y_mc, subj_mc = build_dataset_multiclass(day_embeddings, ema_labels_bin, H=7)
print(X_mc.shape, y_mc.shape)
input_dim = X_mc.shape[1] * X_mc.shape[2]
print(input_dim)

(203, 7, 64) (203,)
448


In [23]:
from torch.utils.data import Dataset, DataLoader

class AffectSequenceDatasetBin(Dataset):
    """
    X: (N, H, 32)  -> per item: (H, 32)
    y: (N,) binary labels (0/1)
    """
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)   # (N, H, 32)
        self.y = torch.tensor(y, dtype=torch.long)      # (N,)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = self.X[idx]          # (H, 32), no flatten
        y = self.y[idx]
        return x, y


In [24]:
import torch.nn as nn
import torch.nn.functional as F

class GRUAffectClassifier(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1, num_classes=2):
        super().__init__()
        self.gru = nn.GRU(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        """
        x: (batch, H, latent_dim)
        """
        out, _ = self.gru(x)         # (batch, H, hidden_dim)
        h_last = out[:, -1, :]       # last time step → (batch, hidden_dim)
        logits = self.fc(h_last)     # (batch, num_classes)
        return logits


In [25]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score
)
import numpy as np
import torch

def evaluate_loso_gru_binary(
    X, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train, y_train = X[train_mask], y[train_mask]
        X_test,  y_test  = X[test_mask],  y[test_mask]

        # we skip if test subject has < 2 samples OR only 1 class (AUC would be degenerate)
        if len(y_test) < 2: #or len(np.unique(y_test)) < 2
            continue

        N_train,H, D = X_train.shape
        N_test = X_test.shape[0]

        X_train_flat = X_train.reshape(N_train, -1)
        X_test_flat = X_test.reshape(N_test, -1)

        scalar= StandardScaler()
        X_train_scaled = scalar.fit_transform(X_train_flat)
        X_test_scaled = scalar.transform(X_test_flat)

        #reshape back to (N, H, latent_dim)
        X_train = X_train_scaled.reshape(N_train, H, D)
        X_test = X_test_scaled.reshape(N_test, H, D)

        train_ds = AffectSequenceDatasetBin(X_train, y_train)
        test_ds  = AffectSequenceDatasetBin(X_test,  y_test)

        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

        # class weights (to handle imbalance)
        class_counts = np.bincount(y_train, minlength=2).astype(float)
        class_counts[class_counts == 0.0] = 1.0
        class_weights = 1.0 / class_counts
        class_weights = class_weights * (2 / class_weights.sum())
        weight_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)

        model = GRUAffectClassifier(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_classes=2
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.CrossEntropyLoss(weight=weight_tensor)

        # Train
        for ep in range(epochs):
            model.train()
            for bx, by in train_loader:
                bx, by = bx.to(device), by.to(device)
                opt.zero_grad()
                logits = model(bx)              # (batch, 2)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # Evaluate
        model.eval()
        all_logits = []
        all_truth = []

        with torch.no_grad():
            for bx, by in test_loader:
                bx = bx.to(device)
                logits = model(bx)
                all_logits.append(logits.cpu())
                all_truth.append(by)

        all_logits = torch.cat(all_logits, dim=0)        # (N_test, 2)
        all_truth = torch.cat(all_truth, dim=0).numpy()  # (N_test,)
        probs = F.softmax(all_logits, dim=1).numpy()     # (N_test, 2)
        preds = probs.argmax(axis=1)                     # (N_test,)
        pos_probs = probs[:, 1]                          # for AUC

        # Metrics
        acc = accuracy_score(all_truth, preds)
        f1_macro = f1_score(all_truth, preds, average="macro", zero_division=0)
        prec_macro = precision_score(all_truth, preds, average="macro", zero_division=0)
        rec_macro = recall_score(all_truth, preds, average="macro", zero_division=0)

        # per-class recall [0, 1]
        label_list = [0, 1]
        rec_per_class = recall_score(
            all_truth, preds,
            average=None,
            labels=label_list,
            zero_division=0
        )

        try:
            auc = roc_auc_score(all_truth, pos_probs)
        except ValueError:
            auc = np.nan

        # Confusion matrix
        cm = confusion_matrix(all_truth, preds, labels=label_list)

        results.append({
            "test_subject": test_subj,
            "n_test": len(all_truth),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "confusion_matrix": cm.tolist(),
            "y_test_unique": list(np.unique(all_truth)),
            "class_counts_train": class_counts.tolist(),
            "class_weights": class_weights.tolist(),
        })

    return pd.DataFrame(results)


In [26]:
set_seed(42)
latent_dim = X_mc.shape[2]   # 64
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_gru = evaluate_loso_gru_binary(
    X_mc, y_mc, subj_mc,
    latent_dim=latent_dim,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_gru)
print("\nAverage metrics across subjects:")
print(loso_gru[["accuracy", "precision_macro", "recall_macro", "f1_macro", "auc"]].mean())


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      21  0.476190         0.416667      0.263158  0.322581   
1         cr003       7  0.285714         0.142857      0.500000  0.222222   
2         cr004       2  1.000000         1.000000      1.000000  1.000000   
3         cr005      15  1.000000         1.000000      1.000000  1.000000   
4         cr006      14  0.642857         0.346154      0.450000  0.391304   
5         cr011       7  0.285714         0.500000      0.142857  0.222222   
6         cr016      10  0.900000         0.500000      0.450000  0.473684   
7         cr031      19  0.684211         0.601190      0.677083  0.592857   
8         cr032      11  0.818182         0.500000      0.409091  0.450000   
9        crs002      17  0.352941         0.500000      0.176471  0.260870   
10       crs007      15  0.000000         0.000000      0.000000  0.000000   
11       crs010      20  0.400000         0.600000      0.647059

Binary prediction using cumulative n-1 days of HR data only

In [27]:
def build_dataset_binary_cumulative_hr(day_embeddings, ema_labels_bin):
    """
    HR-only cumulative history.

    For each subject and each day_n (from the 2nd day onward):

      - Input X_seq: embeddings for ALL past days up to day_{n-1}
            shape (T_i, D), where T_i = number of previous days

      - Target y: EMA_bin at day_n (0/1)

    Returns:
      X_list:  length-N object array, each element is array(T_i, D)
      y:       (N,) int array (0/1)
      subj_ids:(N,) array of subject IDs
    """
    X_list = []
    y_list = []
    subj_list = []

    for subj in day_embeddings:
        if subj not in ema_labels_bin:
            continue

        days = sorted(day_embeddings[subj].keys())

        # start at 7 (need at least seven previous day)
        for i in range(7, len(days)):
            day_n = days[i]           # prediction day
            day_prev = days[i-1]      # previous day
            history_days = days[:i]  # ALL previous days 0..i-1

            # need EMA label for day_n
            if day_n not in ema_labels_bin[subj]:
                continue

            if day_prev not in ema_labels_bin[subj]:
                continue

            try:
                seq = np.vstack([day_embeddings[subj][d] for d in history_days])
                # seq shape: (T_i, D)
            except KeyError:
                continue

            X_list.append(seq)
            y_list.append(ema_labels_bin[subj][day_n])
            subj_list.append(subj)

    X_array = np.array(X_list, dtype=object)       # variable-length
    y = np.array(y_list, dtype=int)
    subj_ids = np.array(subj_list)

    return X_array, y, subj_ids


In [28]:
from torch.utils.data import Dataset, DataLoader

class AffectSequenceDatasetBin_Var(Dataset):
    """
    Variable-length HR-only dataset.

    X_list: object array/list, each element array(T_i, D)
    y:      (N,) 0/1
    """
    def __init__(self, X_list, y):
        self.X = [torch.tensor(x, dtype=torch.float32) for x in X_list]
        self.y = torch.tensor(y, dtype=torch.float32)  # float for BCE

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def collate_varlen_bin(batch):
    """
    batch: list of (seq, y_scalar)
      seq: (T_i, D)
      y_scalar: scalar tensor

    Returns:
      padded:   (B, T_max, D)  left-padded with zeros
      y_batch:  (B,)
      lengths:  (B,) original T_i
    """
    seqs, ys = zip(*batch)
    lengths = [s.shape[0] for s in seqs]
    B = len(seqs)
    D = seqs[0].shape[1]
    T_max = max(lengths)

    padded = torch.zeros(B, T_max, D, dtype=torch.float32)
    for i, s in enumerate(seqs):
        T_i = s.shape[0]
        # left-pad so the most recent day is at the end
        padded[i, T_max - T_i:, :] = s

    y_batch = torch.stack(ys, dim=0)      # (B,)
    lengths = torch.tensor(lengths, dtype=torch.long)

    return padded, y_batch, lengths


In [29]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class GRUAffectBinary(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, 1)  # single logit for BCE

    def forward(self, x, lengths):
        """
        x:       (B, T_max, D)
        lengths: (B,)
        """
        packed = pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.gru(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)  # (B, T_eff, H)

        # last valid timestep
        idx = (lengths - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, out.size(2))
        last_h = out.gather(1, idx).squeeze(1)  # (B, H)

        last_h = self.dropout(last_h)
        logit = self.fc(last_h).squeeze(1)      # (B,)
        return logit


In [30]:
def find_best_threshold(y_true, y_prob, steps=17):
    """
    y_true: (N,) binary 0/1
    y_prob: (N,) predicted probability of class 1
    Returns: best_threshold, best_f1
    """
    best_t = 0.5
    best_f1 = -1.0
    for t in np.linspace(0.1, 0.9, steps):
        y_pred = (y_prob >= t).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
    return best_t, best_f1


In [31]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
    roc_auc_score,
    confusion_matrix
)

def evaluate_loso_gru_binary_cumulative_hr(
    X_list, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    """
    X_list: object array/list of length N, each element (T_i, D)
    y:      (N,) 0/1
    subj_ids: (N,)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train_list = X_list[train_mask]
        X_test_list  = X_list[test_mask]
        y_train      = y[train_mask]
        y_test       = y[test_mask]

        # need at least 2 test samples
        if len(y_test) < 2:
            continue

        # NORMALIZATION (on training only)
        # compute mean/std over all timepoints and dims in training
        all_train_frames = np.vstack(X_train_list)  # (sum T_i, D)
        mean = all_train_frames.mean(axis=0, keepdims=True)  # (1, D)
        std = all_train_frames.std(axis=0, keepdims=True)
        std[std == 0] = 1.0

        def normalize_seq_list(XL):
            out = []
            for arr in XL:
                arr_norm = (arr - mean) / std
                out.append(arr_norm.astype(np.float32))
            return np.array(out, dtype=object)

        X_train_list_norm = normalize_seq_list(X_train_list)
        X_test_list_norm  = normalize_seq_list(X_test_list)


        # datasets/loaders
        train_ds = AffectSequenceDatasetBin_Var(X_train_list_norm, y_train)
        test_ds  = AffectSequenceDatasetBin_Var(X_test_list_norm,  y_test)

        train_loader = DataLoader(
            train_ds, batch_size=batch_size, shuffle=True,
            collate_fn=collate_varlen_bin
        )
        test_loader = DataLoader(
            test_ds, batch_size=batch_size, shuffle=False,
            collate_fn=collate_varlen_bin
        )

        # class imbalance for BCE pos_weight
        class_counts = np.bincount(y_train.astype(int), minlength=2).astype(float)
        if class_counts[1] == 0:
            pos_weight_val = 1.0
        else:
            pos_weight_val = class_counts[0] / class_counts[1]
        pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32, device=device)

        # latent_dim from first sequence
        latent_dim_here = X_train_list_norm[0].shape[1]

        model = GRUAffectBinary(
            latent_dim=latent_dim_here,
            hidden_dim=hidden_dim,
            num_layers=num_layers
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        # TRAIN
        for ep in range(epochs):
            model.train()
            for bx, by, lengths in train_loader:
                bx = bx.to(device)         # (B, T_max, D)
                by = by.to(device)         # (B,) float
                lengths = lengths.to(device)

                opt.zero_grad()
                logits = model(bx, lengths)  # (B,)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # TRAIN PREDICTIONS FOR THRESHOLD
        model.eval()
        train_logits_all = []
        train_truth_all  = []

        with torch.no_grad():
            for bx, by, lengths in train_loader:
                bx = bx.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths)
                train_logits_all.append(logits.cpu())
                train_truth_all.append(by)

        train_logits_all = torch.cat(train_logits_all, dim=0).numpy()
        train_truth_all  = torch.cat(train_truth_all, dim=0).numpy()
        train_probs_all  = 1.0 / (1.0 + np.exp(-train_logits_all))  # sigmoid

        best_thr, best_f1_train = find_best_threshold(train_truth_all, train_probs_all)

        # TEST PREDICTIONS
        test_logits_all = []
        test_truth_all  = []

        with torch.no_grad():
            for bx, by, lengths in test_loader:
                bx = bx.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths)
                test_logits_all.append(logits.cpu())
                test_truth_all.append(by)

        test_logits_all = torch.cat(test_logits_all, dim=0).numpy()
        test_truth_all  = torch.cat(test_truth_all, dim=0).numpy()
        test_probs_all  = 1.0 / (1.0 + np.exp(-test_logits_all))

        preds = (test_probs_all >= best_thr).astype(int)
        y_int = test_truth_all.astype(int)

        # metrics
        acc = accuracy_score(y_int, preds)
        f1_macro = f1_score(y_int, preds, average="macro", zero_division=0)
        prec_macro = precision_score(y_int, preds, average="macro", zero_division=0)
        rec_macro = recall_score(y_int, preds, average="macro", zero_division=0)

        rec_per_class = recall_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )
        prec_per_class = precision_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )

        try:
            auc = roc_auc_score(y_int, test_probs_all)
        except ValueError:
            auc = np.nan

        cm = confusion_matrix(y_int, preds, labels=[0, 1])
        cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
        cm_norm = np.nan_to_num(cm_norm)

        results.append({
            "test_subject": test_subj,
            "n_test": len(y_int),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "precision_per_class": prec_per_class.tolist(),
            "confusion_matrix": cm_norm.tolist(),
            "y_test_unique": list(np.unique(y_int)),
            "class_counts_train": class_counts.tolist(),
            "pos_weight": float(pos_weight_val),
            "best_threshold": float(best_thr),
            "train_best_f1": float(best_f1_train),
        })

    return pd.DataFrame(results)


In [32]:
# Build cumulative HR-only dataset
X_hr_cum, y_hr_cum, subj_hr_cum = build_dataset_binary_cumulative_hr(
    day_embeddings, ema_labels_bin
)

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_hr_cum = evaluate_loso_gru_binary_cumulative_hr(
    X_hr_cum, y_hr_cum, subj_hr_cum,
    latent_dim=X_hr_cum[0].shape[1],   # D from first sequence
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_hr_cum)
print("\nAverage metrics:")
print(loso_hr_cum[["accuracy", "precision_macro", "recall_macro", "f1_macro", "auc"]].mean())


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-55763866.py:171: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-55763866.py:171: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-55763866.py:171: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      19  0.315789         0.566667      0.617647  0.308123   
1         cr003       7  0.428571         0.458333      0.450000  0.416667   
2         cr005      14  0.357143         0.500000      0.178571  0.263158   
3         cr006       5  0.400000         0.200000      0.500000  0.285714   
4         cr011       4  0.000000         0.000000      0.000000  0.000000   
5         cr016       4  1.000000         1.000000      1.000000  1.000000   
6         cr031       8  0.250000         0.500000      0.125000  0.200000   
7         cr032       9  1.000000         1.000000      1.000000  1.000000   
8        crs002      14  0.785714         0.500000      0.392857  0.440000   
9        crs007      15  0.266667         0.500000      0.133333  0.210526   
10       crs010      17  0.529412         0.600000      0.733333  0.484848   
11       crs011      13  0.153846         0.500000      0.076923

GRU With Previous EMA as input feature with cumulative n-1 days


In [33]:
import numpy as np

def build_dataset_binary_cumulative_with_prev_ema(day_embeddings, ema_labels_bin):
    """
    For each subject and each day_n (from the 2nd day onward):

      - Input X_seq: embeddings for ALL past days up to day_{n-1}
            shape (T_i, D), where T_i = number of previous days

      - Extra feature prev_ema: EMA_bin at day_{n-1}  (0/1)

      - Target y: EMA_bin at day_n (0/1)

    Returns:
      X_list:  length-N list of arrays, each of shape (T_i, D)
      prev_ema: (N,) float array (0/1)
      y:        (N,) int array (0/1)
      subj_ids: (N,) array of subject IDs
    """
    X_list = []
    prev_ema_list = []
    y_list = []
    subj_list = []

    for subj in day_embeddings:
        # must have EMA labels for this subject
        if subj not in ema_labels_bin:
            continue

        # sorted days for which we have embeddings
        days = sorted(day_embeddings[subj].keys())

        # start from i = 1 (second day), since we need a previous day
        for i in range(7, len(days)):
            day_n = days[i]        # prediction day
            day_prev = days[i-1]   # immediate previous day
            history_days = days[:i]  # ALL days before day_n: days[0..i-1]

            # need EMA label at day_n and day_prev
            if day_n not in ema_labels_bin[subj]:
                continue
            if day_prev not in ema_labels_bin[subj]:
                continue

            # build sequence of embeddings for all previous days
            try:
                seq = np.vstack([day_embeddings[subj][d] for d in history_days])
                # seq shape: (T_i, D)
            except KeyError:
                # if some day is missing in embeddings, skip this sample
                continue

            X_list.append(seq)
            y_list.append(ema_labels_bin[subj][day_n])      # label for day_n
            prev_ema_list.append(ema_labels_bin[subj][day_prev])  # EMA of day_{n-1}
            subj_list.append(subj)

    # variable-length sequences → keep X_list as a Python list (or object array)
    X_array = np.array(X_list, dtype=object)  # each element is (T_i, D)
    prev_ema = np.array(prev_ema_list, dtype=np.float32)
    y = np.array(y_list, dtype=int)
    subj_ids = np.array(subj_list)

    return X_array, prev_ema, y, subj_ids


In [34]:
class AffectSequenceDatasetBinWithEMA_Var(Dataset):
    """
    Variable-length version.

    X_list: list (or array) of length N,
            each element is a NumPy array of shape (T_i, D).

    prev_ema: (N,) array of 0/1 (yesterday's EMA)
    y:        (N,) array of 0/1 (today's EMA)
    """
    def __init__(self, X_list, prev_ema, y):
        # store sequences as a list of tensors with varying T_i
        self.X = [torch.tensor(x, dtype=torch.float32) for x in X_list]
        self.prev_ema = torch.tensor(prev_ema, dtype=torch.float32)  # (N,)
        self.y = torch.tensor(y, dtype=torch.float32)                # (N,)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # returns (seq, prev_ema_scalar, label)
        return self.X[idx], self.prev_ema[idx], self.y[idx]


In [35]:
def collate_varlen_bin_with_ema(batch):
    """
    batch: list of (seq, prev_ema_scalar, y_scalar)
      seq: (T_i, D)
      prev_ema_scalar: scalar tensor
      y_scalar: scalar tensor

    Returns:
      padded:        (B, T_max, D)  left-padded with zeros
      prev_ema_batch:(B,)
      y_batch:       (B,)
      lengths:       (B,) original T_i
    """
    import torch

    seqs, prev_emas, ys = zip(*batch)
    lengths = [s.shape[0] for s in seqs]
    B = len(seqs)
    D = seqs[0].shape[1]
    T_max = max(lengths)

    padded = torch.zeros(B, T_max, D, dtype=torch.float32)
    for i, s in enumerate(seqs):
        T_i = s.shape[0]
        # left-pad so that the most recent day is at the END of the sequence
        padded[i, T_max - T_i:, :] = s

    prev_ema_batch = torch.stack(prev_emas, dim=0)  # (B,)
    y_batch = torch.stack(ys, dim=0)                # (B,)
    lengths = torch.tensor(lengths, dtype=torch.long)

    return padded, prev_ema_batch, y_batch, lengths


In [36]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch

class GRUAffectBinaryWithEMA(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim + 1, 1)  # +1 for prev_ema

    def forward(self, x, lengths, prev_ema):
        # x: (B, T_max, D)
        # lengths: (B,)
        # prev_ema: (B,)
        packed = pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.gru(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)  # (B, T_eff, H)

        # last valid timestep
        idx = (lengths - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, out.size(2))
        last_h = out.gather(1, idx).squeeze(1)  # (B, H)

        last_h = self.dropout(last_h)
        prev_ema = prev_ema.unsqueeze(1)        # (B, 1)
        h_cat = torch.cat([last_h, prev_ema], dim=1)  # (B, H+1)

        logit = self.fc(h_cat).squeeze(1)       # (B,)
        return logit



In [37]:
import numpy as np
from sklearn.metrics import f1_score

def find_best_threshold(y_true, y_prob, steps=17):
    """
    y_true: (N,) binary 0/1
    y_prob: (N,) predicted probability of class 1
    steps: number of thresholds between 0.1 and 0.9 to scan

    Returns: best_threshold, best_f1
    """
    best_t = 0.5
    best_f1 = -1.0
    for t in np.linspace(0.1, 0.9, steps):
        y_pred = (y_prob >= t).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
    return best_t, best_f1


In [38]:
set_seed(42)
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix
)

def evaluate_loso_gru_binary_with_prev_ema_threshold_varlen(
    X_list, prev_ema, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    """
    X_list: np.array (dtype=object) of length N, each element array(T_i, D)
    prev_ema: (N,)
    y: (N,)  0/1
    subj_ids: (N,)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train_list = X_list[train_mask]
        prev_train   = prev_ema[train_mask]
        y_train      = y[train_mask]

        X_test_list  = X_list[test_mask]
        prev_test    = prev_ema[test_mask]
        y_test       = y[test_mask]

        # need at least 2 test samples
        if len(y_test) < 2:
            continue

        # datasets / loaders
        train_ds = AffectSequenceDatasetBinWithEMA_Var(X_train_list, prev_train, y_train)
        test_ds  = AffectSequenceDatasetBinWithEMA_Var(X_test_list,  prev_test,  y_test)

        train_loader = DataLoader(
            train_ds, batch_size=batch_size, shuffle=True,
            collate_fn=collate_varlen_bin_with_ema
        )
        test_loader  = DataLoader(
            test_ds, batch_size=batch_size, shuffle=False,
            collate_fn=collate_varlen_bin_with_ema
        )

        # class imbalance (pos_weight for BCE)
        class_counts = np.bincount(y_train.astype(int), minlength=2).astype(float)
        # class_counts[0] = #neg, class_counts[1] = #pos
        if class_counts[1] == 0:
            pos_weight_val = 1.0
        else:
            pos_weight_val = class_counts[0] / class_counts[1]

        pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32, device=device)

        # model / loss
        # infer latent_dim from first sequence
        if isinstance(X_list[0], np.ndarray):
            latent_dim_here = X_list[0].shape[1]
        else:
            latent_dim_here = latent_dim

        model = GRUAffectBinaryWithEMA(
            latent_dim=latent_dim_here,
            hidden_dim=hidden_dim,
            num_layers=num_layers
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        #to track loss per epoch for subject
        train_loss_per_epoch = []
        test_loss_per_epoch = []

        # TRAIN
        for ep in range(epochs):
            model.train()
            total_train_loss = 0.0
            n_train = 0

            for bx, bprev, by, lengths in train_loader:
                bx = bx.to(device)             # (B, T_max, D)
                bprev = bprev.to(device)       # (B,)
                by = by.to(device)             # (B,) float 0/1
                lengths = lengths.to(device)

                opt.zero_grad()
                logits = model(bx, lengths, bprev)  # (B,)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

                total_train_loss += loss.item() * by.size(0)
                n_train += by.size(0)


            avg_train_loss = total_train_loss / max(n_train, 1)
            train_loss_per_epoch.append(avg_train_loss)

            # TEST (VAL) LOSS
            model.eval()
            total_test_loss = 0.0
            n_test_loss = 0

            with torch.no_grad():
                for bx, bprev, by, lengths in test_loader:
                    bx = bx.to(device)
                    bprev = bprev.to(device)
                    lengths = lengths.to(device)
                    by = by.to(device)

                    logits = model(bx, lengths, bprev)
                    loss = loss_fn(logits, by)

                    total_test_loss += loss.item() * by.size(0)
                    n_test_loss += by.size(0)

            avg_test_loss = total_test_loss / max(n_test_loss, 1)
            test_loss_per_epoch.append(avg_test_loss)


        # TRAIN PREDICTIONS FOR THRESHOLD TUNING
        model.eval()
        train_logits_all = []
        train_truth_all  = []

        with torch.no_grad():
            for bx, bprev, by, lengths in train_loader:
                bx = bx.to(device)
                bprev = bprev.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths, bprev)  # (B,)
                train_logits_all.append(logits.cpu())
                train_truth_all.append(by)

        train_logits_all = torch.cat(train_logits_all, dim=0).numpy()   # (N_train,)
        train_truth_all  = torch.cat(train_truth_all, dim=0).numpy()    # (N_train,)
        train_probs_all  = 1.0 / (1.0 + np.exp(-train_logits_all))      # sigmoid

        best_thr, best_f1_train = find_best_threshold(train_truth_all, train_probs_all)

        # TEST PREDICTIONS
        test_logits_all = []
        test_truth_all  = []

        with torch.no_grad():
            for bx, bprev, by, lengths in test_loader:
                bx = bx.to(device)
                bprev = bprev.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths, bprev)
                test_logits_all.append(logits.cpu())
                test_truth_all.append(by)

        test_logits_all = torch.cat(test_logits_all, dim=0).numpy()  # (N_test,)
        test_truth_all  = torch.cat(test_truth_all, dim=0).numpy()   # (N_test,)
        test_probs_all  = 1.0 / (1.0 + np.exp(-test_logits_all))

        preds = (test_probs_all >= best_thr).astype(int)
        y_int = test_truth_all.astype(int)

        # metrics
        acc = accuracy_score(y_int, preds)
        f1_macro = f1_score(y_int, preds, average="macro", zero_division=0)
        prec_macro = precision_score(y_int, preds, average="macro", zero_division=0)
        rec_macro = recall_score(y_int, preds, average="macro", zero_division=0)

        rec_per_class = recall_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )
        prec_per_class = precision_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )

        try:
            auc = roc_auc_score(y_int, test_probs_all)
        except ValueError:
            auc = np.nan

        cm = confusion_matrix(y_int, preds, labels=[0, 1])
        cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
        cm_norm = np.nan_to_num(cm_norm)

        results.append({
            "test_subject": test_subj,
            "n_test": len(y_int),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "precision_per_class": prec_per_class.tolist(),
            "confusion_matrix": cm_norm.tolist(),
            "y_test_unique": list(np.unique(y_int)),
            "class_counts_train": class_counts.tolist(),
            "pos_weight": float(pos_weight_val),
            "best_threshold": float(best_thr),
            "train_best_f1": float(best_f1_train),
            "train_loss_curve": train_loss_per_epoch,
            "test_loss_curve": test_loss_per_epoch,
        })

    return pd.DataFrame(results)




In [39]:
set_seed(42)
X_bin, prev_bin, y_bin, subj_bin = build_dataset_binary_cumulative_with_prev_ema(
    day_embeddings, ema_labels_bin
)


device = "cuda" if torch.cuda.is_available() else "cpu"

loso_bin_cum = evaluate_loso_gru_binary_with_prev_ema_threshold_varlen(
    X_bin, prev_bin, y_bin, subj_bin,
    latent_dim=X_bin[0].shape[1],   # D from first sequence
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_bin_cum)

print("\nAverage metrics:")
print(loso_bin_cum[["accuracy", "precision_macro", "recall_macro", "f1_macro", "auc"]].mean())



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-3135434405.py:199: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-3135434405.py:199: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-3135434405.py:199: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype(

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      19  0.315789         0.566667      0.617647  0.308123   
1         cr003       7  0.571429         0.333333      0.400000  0.363636   
2         cr005      14  0.500000         0.500000      0.250000  0.333333   
3         cr006       5  0.600000         0.583333      0.583333  0.583333   
4         cr011       4  0.250000         0.500000      0.125000  0.200000   
5         cr016       4  0.500000         0.500000      0.250000  0.333333   
6         cr031       8  0.500000         0.500000      0.250000  0.333333   
7         cr032       9  0.444444         0.500000      0.222222  0.307692   
8        crs002      14  0.214286         0.500000      0.107143  0.176471   
9        crs007      15  0.333333         0.500000      0.166667  0.250000   
10       crs010      17  0.588235         0.611111      0.766667  0.529644   
11       crs011      13  0.769231         0.500000      0.384615

In [41]:
train_curves = np.array(loso_bin_cum["train_loss_curve"].tolist())  # shape (S, E)
test_curves  = np.array(loso_bin_cum["test_loss_curve"].tolist())   # shape (S, E)

# Compute mean across subjects
train_mean = np.nanmean(train_curves, axis=0)
test_mean  = np.nanmean(test_curves, axis=0)

epochs = np.arange(1, len(train_mean)+1)

In [1]:
#plt.figure(figsize=(8,5))
#plt.plot(epochs, train_mean, label="Train Loss (avg across subjects)", linewidth=2)
#plt.plot(epochs, test_mean,  label="Test Loss (avg across subjects)", linewidth=2)

#plt.xlabel("Epoch")
#plt.ylabel("Loss (BCEWithLogits)")
#plt.title("Global Train/Test Loss Curves – GRU (cumulative + prev EMA)")
#plt.legend()
#plt.grid(True)
#plt.show()

HR + Prev EMA with fixed H

In [44]:
def build_dataset_binary_with_prev_ema(day_embeddings, ema_labels_bin, H=7):
    """
    For each subject:
      - Input X: (H, 32) embeddings for days [n-H, ..., n-1]
      - Extra feature prev_ema: EMA_bin at day (n-1)
      - Target y: EMA_bin at day n
    Returns:
      X: (N, H, 32)
      prev_ema: (N,)
      y: (N,)
      subj_ids: (N,)
    """
    X_list, prev_ema_list, y_list, subj_list = [], [], [], []

    for subj in day_embeddings:
        if subj not in ema_labels_bin:
            continue

        days = sorted(day_embeddings[subj].keys())

        for i in range(H, len(days)):
            day_n = days[i]          # prediction day
            prev_days = days[i-H:i]  # H previous days
            day_prev = days[i-1]     # immediate previous day

            # need label at day n and day_prev
            if day_n not in ema_labels_bin[subj]:
                continue
            if day_prev not in ema_labels_bin[subj]:
                continue

            try:
                seq = np.vstack([day_embeddings[subj][d] for d in prev_days])  # (H, 32)
            except KeyError:
                continue

            X_list.append(seq)
            y_list.append(ema_labels_bin[subj][day_n])
            prev_ema_list.append(ema_labels_bin[subj][day_prev])
            subj_list.append(subj)

    X = np.array(X_list, dtype=np.float32)
    prev_ema = np.array(prev_ema_list, dtype=np.float32)
    y = np.array(y_list, dtype=int)
    subj_ids = np.array(subj_list)

    return X, prev_ema, y, subj_ids


In [45]:
H = 7
X_ema, prev_ema_arr, y_ema, subj_ema = build_dataset_binary_with_prev_ema(
    day_embeddings, ema_labels_bin, H=H
)
print(X_ema.shape, prev_ema_arr.shape, y_ema.shape)


(154, 7, 64) (154,) (154,)


In [46]:
class AffectSequenceDatasetBinWithEMA(Dataset):
    """
    X: (N, H, 32)
    prev_ema: (N,) scalar 0/1 (yesterday's affect)
    y: (N,)
    """
    def __init__(self, X, prev_ema, y):
        self.X = torch.tensor(X, dtype=torch.float32)          # (N, H, 32)
        self.prev_ema = torch.tensor(prev_ema, dtype=torch.float32)  # (N,)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.prev_ema[idx], self.y[idx]


In [47]:
class GRUAffectClassifierWithEMA(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1, num_classes=2):
        super().__init__()
        self.gru = nn.GRU(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim + 1, num_classes)  # +1 for prev_ema scalar

    def forward(self, x, prev_ema):
        """
        x: (batch, H, latent_dim)
        prev_ema: (batch,) scalar 0/1
        """
        out, _ = self.gru(x)         # (batch, H, hidden_dim)
        h_last = out[:, -1, :]       # (batch, hidden_dim)
        prev_ema = prev_ema.unsqueeze(-1)    # (batch, 1)
        feat = torch.cat([h_last, prev_ema], dim=1)  # (batch, hidden_dim + 1)
        logits = self.fc(feat)                   # (batch, num_classes)
        return logits


In [48]:
def evaluate_loso_gru_binary_with_prev_ema(
    X, prev_ema, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train, prev_train, y_train = X[train_mask], prev_ema[train_mask], y[train_mask]
        X_test,  prev_test,  y_test  = X[test_mask],  prev_ema[test_mask],  y[test_mask]

        if len(y_test) < 2: #or len(np.unique(y_test)) < 2:
            continue



        train_ds = AffectSequenceDatasetBinWithEMA(X_train, prev_train, y_train)
        test_ds  = AffectSequenceDatasetBinWithEMA(X_test,  prev_test,  y_test)

        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

        # class weights
        class_counts = np.bincount(y_train, minlength=2).astype(float)
        class_counts[class_counts == 0.0] = 1.0
        class_weights = 1.0 / class_counts
        class_weights = class_weights * (2 / class_weights.sum())
        weight_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)

        model = GRUAffectClassifierWithEMA(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_classes=2
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.CrossEntropyLoss(weight=weight_tensor)

        # Train
        for ep in range(epochs):
            model.train()
            for bx, bprev, by in train_loader:
                bx, bprev, by = bx.to(device), bprev.to(device), by.to(device)
                opt.zero_grad()
                logits = model(bx, bprev)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # Evaluate
        model.eval()
        all_logits = []
        all_truth = []

        with torch.no_grad():
            for bx, bprev, by in test_loader:
                bx, bprev = bx.to(device), bprev.to(device)
                logits = model(bx, bprev)
                all_logits.append(logits.cpu())
                all_truth.append(by)

        all_logits = torch.cat(all_logits, dim=0)
        all_truth = torch.cat(all_truth, dim=0).numpy()
        probs = F.softmax(all_logits, dim=1).numpy()
        preds = probs.argmax(axis=1)
        pos_probs = probs[:, 1]

        acc = accuracy_score(all_truth, preds)
        f1_macro = f1_score(all_truth, preds, average="macro", zero_division=0)
        prec_macro = precision_score(all_truth, preds, average="macro", zero_division=0)
        rec_macro = recall_score(all_truth, preds, average="macro", zero_division=0)

        label_list = [0, 1]
        rec_per_class = recall_score(
            all_truth, preds,
            average=None,
            labels=label_list,
            zero_division=0
        )
        prec_per_class = precision_score(
            all_truth, preds,
            average=None,
            labels=label_list,
            zero_division=0
        )

        try:
            auc = roc_auc_score(all_truth, pos_probs)
        except ValueError:
            auc = np.nan

        cm = confusion_matrix(all_truth, preds, labels=label_list)
        cm_norm = cm.astype('float') / cm.sum(axis = 1, keepdims=True)
        cm_norm = np.nan_to_num(cm_norm)

        results.append({
            "test_subject": test_subj,
            "n_test": len(all_truth),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "precision_per_class": prec_per_class.tolist(),
            "confusion_matrix": cm_norm.tolist(),
            "y_test_unique": list(np.unique(all_truth)),
            "class_counts_train": class_counts.tolist(),
            "class_weights": class_weights.tolist(),
        })


    return pd.DataFrame(results)


In [49]:
set_seed(42)
H = 7
X_ema, prev_ema_arr, y_ema, subj_ema = build_dataset_binary_with_prev_ema(
    day_embeddings, ema_labels_bin, H=H
)
latent_dim = X_ema.shape[2]
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_gru_ema = evaluate_loso_gru_binary_with_prev_ema(
    X_ema, prev_ema_arr, y_ema, subj_ema,
    latent_dim=latent_dim,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_gru_ema)
print("\nAverage metrics across subjects (HR + prev EMA):")
print(loso_gru_ema[["accuracy", "precision_macro", "recall_macro", "f1_macro", "auc"]].mean())


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-4125017975.py:105: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis = 1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-4125017975.py:105: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis = 1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-4125017975.py:105: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.ast

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      19  0.473684         0.409091      0.264706  0.321429   
1         cr003       7  0.714286         0.357143      0.500000  0.416667   
2         cr005      14  0.357143         0.500000      0.178571  0.263158   
3         cr006       5  0.400000         0.416667      0.416667  0.400000   
4         cr011       4  0.250000         0.500000      0.125000  0.200000   
5         cr016       4  1.000000         1.000000      1.000000  1.000000   
6         cr031       8  0.625000         0.500000      0.312500  0.384615   
7         cr032       9  0.777778         0.500000      0.388889  0.437500   
8        crs002      14  0.642857         0.500000      0.321429  0.391304   
9        crs007      15  0.266667         0.500000      0.133333  0.210526   
10       crs010      17  0.411765         0.583333      0.666667  0.392857   
11       crs011      13  0.230769         0.500000      0.115385

Multi-label Multi Neurons fixed H


In [50]:
df_ema_ml = df_ema.copy()

# Making sure they are ints 0/1
df_ema_ml["Positive"] = df_ema_ml["Positive"].astype(int)
df_ema_ml["Negative"] = df_ema_ml["Negative"].astype(int)

# Building dict: ema_labels_multi[subj][date] = np.array([neg, pos])
ema_labels_multi = {}

for subj, df_s in df_ema_ml.groupby("User_ID"):
    ema_labels_multi[subj] = {}
    for _, row in df_s.iterrows():
        d = pd.to_datetime(row["Date_only"]).date()
        neg = int(row["Negative"])
        pos = int(row["Positive"])
        ema_labels_multi[subj][d] = np.array([neg, pos], dtype=np.float32)


In [51]:
def build_dataset_multilabel(day_embeddings, ema_labels_multi, H=7):
    """
    Returns:
      X: (N_samples, H, 32)
      y: (N_samples, 2)   # [neg, pos]
      subj_ids: (N_samples,)
    """
    X_list, y_list, subj_list = [], [], []

    for subj in day_embeddings:
        if subj not in ema_labels_multi:
            continue

        days = sorted(day_embeddings[subj].keys())

        for i in range(H, len(days)):
            day_n = days[i]          # prediction day
            prev_days = days[i-H:i]  # H previous days

            if day_n not in ema_labels_multi[subj]:
                continue

            # Make sure we have embeddings for all previous days
            try:
                seq = np.vstack([day_embeddings[subj][d] for d in prev_days])  # (H, 32)
            except KeyError:
                continue

            y_vec = ema_labels_multi[subj][day_n]   # (2,) [neg, pos]

            X_list.append(seq)
            y_list.append(y_vec)
            subj_list.append(subj)

    X = np.array(X_list, dtype=np.float32)     # (N, H, 32)
    y = np.array(y_list, dtype=np.float32)     # (N, 2)
    subj_ids = np.array(subj_list)

    return X, y, subj_ids

H = 7  # same horizon you used before
X_ml, y_ml, subj_ml = build_dataset_multilabel(day_embeddings, ema_labels_multi, H=H)
print(X_ml.shape, y_ml.shape, subj_ml.shape)
# e.g. (N, 2, 32), (N, 2), (N,)


(242, 7, 64) (242, 2) (242,)


In [52]:
from torch.utils.data import Dataset

class AffectSequenceDatasetMultiLabel(Dataset):
    """
    X: (N, H, 32)
    y: (N, 2)  -> [neg, pos]
    """
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)  # float for BCE

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [53]:
import torch.nn as nn

class GRUAffectMultiLabel(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1, num_labels=2):
        super().__init__()
        self.gru = nn.GRU(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, num_labels)  # 2 labels: [neg, pos]

    def forward(self, x):
        """
        x: (batch, H, latent_dim)
        returns: logits of shape (batch, 2)
        """
        out, _ = self.gru(x)        # (batch, H, hidden_dim)
        h_last = out[:, -1, :]      # (batch, hidden_dim)
        logits = self.fc(h_last)    # (batch, 2)
        return logits


In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix
)
import torch.nn.functional as F

def confusion_normalized(cm):
    """
    Row-normalize a confusion matrix.
    cm: (2,2) array
    returns (2,2) float array
    """
    cm = cm.astype(float)
    row_sums = cm.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1.0
    return cm / row_sums


def evaluate_loso_gru_multilabel(
    X, y, subj_ids,
    latent_dim=None,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    """
    X: (N, H, latent_dim)
    y: (N, 2)  -> [neg, pos]
    subj_ids: (N,)
    """
    import numpy as np
    import torch

    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    if latent_dim is None:
        latent_dim = X.shape[2]

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train, y_train = X[train_mask], y[train_mask]
        X_test,  y_test  = X[test_mask],  y[test_mask]

        if len(y_test) < 2:
            continue

        #  NORMALIZE (train only)
        N_train, H, D = X_train.shape
        N_test = X_test.shape[0]

        X_train_flat = X_train.reshape(N_train, -1)
        X_test_flat  = X_test.reshape(N_test,  -1)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_flat)
        X_test_scaled  = scaler.transform(X_test_flat)

        X_train = X_train_scaled.reshape(N_train, H, D)
        X_test  = X_test_scaled.reshape(N_test,  H, D)


        # Datasets
        train_ds = AffectSequenceDatasetMultiLabel(X_train, y_train)
        test_ds  = AffectSequenceDatasetMultiLabel(X_test,  y_test)

        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

        # class weights (pos_weight)
        pos_counts = y_train.sum(axis=0)
        neg_counts = len(y_train) - pos_counts

        # avoid zero division
        pos_counts = np.where(pos_counts == 0, 1.0, pos_counts)
        neg_counts = np.where(neg_counts == 0, 1.0, neg_counts)

        raw_pw = neg_counts / pos_counts
        alpha = 0.5
        pos_weight = 1.0 + alpha * (raw_pw - 1.0)

        pos_weight_t = torch.tensor(pos_weight, dtype=torch.float32, device=device)

        # Model
        model = GRUAffectMultiLabel(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_labels=2
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight_t)

        # TRAIN
        for ep in range(epochs):
            model.train()
            for bx, by in train_loader:
                bx, by = bx.to(device), by.to(device)
                opt.zero_grad()
                logits = model(bx)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # TRAIN PRED FOR THRESHOLD TUNING
        model.eval()
        train_logits = []
        train_truth = []

        with torch.no_grad():
            for bx, by in train_loader:
                bx = bx.to(device)
                logits = model(bx)
                train_logits.append(logits.cpu())
                train_truth.append(by)

        train_logits = torch.cat(train_logits, dim=0)
        train_truth = torch.cat(train_truth, dim=0).numpy()
        train_probs = torch.sigmoid(train_logits).numpy()

        # Tune thresholds (per label)
        def find_best_threshold(y_true, y_prob, steps=17):
            best_t = 0.5
            best_f1 = 0
            for t in np.linspace(0.1, 0.9, steps):
                y_pred = (y_prob >= t).astype(int)
                f1 = f1_score(y_true, y_pred, zero_division=0)
                if f1 > best_f1:
                    best_f1 = f1
                    best_t = t
            return best_t

        thr_neg = find_best_threshold(train_truth[:,0], train_probs[:,0])
        thr_pos = find_best_threshold(train_truth[:,1], train_probs[:,1])

        # TEST PRED
        test_logits = []
        test_truth = []

        with torch.no_grad():
            for bx, by in test_loader:
                bx = bx.to(device)
                logits = model(bx)
                test_logits.append(logits.cpu())
                test_truth.append(by)

        test_logits = torch.cat(test_logits, dim=0)
        test_truth = torch.cat(test_truth, dim=0).numpy()

        probs = torch.sigmoid(test_logits).numpy()
        preds = np.zeros_like(probs, dtype=int)

        preds[:,0] = (probs[:,0] >= thr_neg).astype(int)
        preds[:,1] = (probs[:,1] >= thr_pos).astype(int)

        metrics = {}
        raw_cm_dict = {}
        norm_cm_dict = {}

        # Per label metrics + confusion matrices
        for j, name in enumerate(["neg", "pos"]):
            y_true_j = test_truth[:,j]
            y_pred_j = preds[:,j]

            # Metrics
            metrics[f"acc_{name}"]  = accuracy_score(y_true_j, y_pred_j)
            metrics[f"prec_{name}"] = precision_score(y_true_j, y_pred_j, zero_division=0)
            metrics[f"rec_{name}"]  = recall_score(y_true_j, y_pred_j, zero_division=0)
            metrics[f"f1_{name}"]   = f1_score(y_true_j, y_pred_j, zero_division=0)

            # Confusion matrix (2x2)
            cm = confusion_matrix(y_true_j, y_pred_j, labels=[0,1])
            cm_norm = confusion_normalized(cm)

            raw_cm_dict[f"confusion_{name}"] = cm.tolist()
            norm_cm_dict[f"confusion_norm_{name}"] = cm_norm.tolist()

        # Macro
        metrics["acc_macro"]  = 0.5*(metrics["acc_neg"]+metrics["acc_pos"])
        metrics["prec_macro"] = 0.5*(metrics["prec_neg"]+metrics["prec_pos"])
        metrics["rec_macro"]  = 0.5*(metrics["rec_neg"]+metrics["rec_pos"])
        metrics["f1_macro"]   = 0.5*(metrics["f1_neg"]+metrics["f1_pos"])

        results.append({
            "test_subject": test_subj,
            "n_test": len(test_truth),
            **metrics,
            **raw_cm_dict,
            **norm_cm_dict,
            "pos_weight": pos_weight.tolist(),
            "thr_neg": float(thr_neg),
            "thr_pos": float(thr_pos)
        })

    return pd.DataFrame(results)



In [55]:
set_seed(42)
latent_dim = X_ml.shape[2]
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_ml = evaluate_loso_gru_multilabel(
    X_ml, y_ml, subj_ml,
    latent_dim=latent_dim,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_ml)
print("\nAverage metrics:")
print(loso_ml[["acc_macro", "prec_macro", "rec_macro", "f1_macro"]].mean())


   test_subject  n_test   acc_neg  prec_neg   rec_neg    f1_neg   acc_pos  \
0         cr001      22  0.590909  0.000000  0.000000  0.000000  0.727273   
1         cr002      19  0.473684  0.000000  0.000000  0.000000  0.578947   
2         cr003       7  0.428571  0.333333  1.000000  0.500000  0.428571   
3         cr004       3  0.666667  0.666667  1.000000  0.800000  1.000000   
4         cr005      16  0.812500  0.000000  0.000000  0.000000  0.687500   
5         cr006      14  0.500000  0.000000  0.000000  0.000000  0.500000   
6         cr011      11  0.636364  1.000000  0.636364  0.777778  0.545455   
7         cr016      10  1.000000  0.000000  0.000000  0.000000  0.600000   
8         cr031      24  0.833333  0.904762  0.904762  0.904762  0.416667   
9         cr032      13  0.692308  0.500000  0.500000  0.500000  0.923077   
10       crs002      22  0.454545  1.000000  0.454545  0.625000  0.318182   
11       crs007      15  0.733333  0.000000  0.000000  0.000000  0.000000   

Multi-label with cumulative n-1 days HR data

In [56]:
def build_dataset_cumulative_multilabel(day_embeddings, ema_labels_multi):
    """
    For each subject and each day_n (from 2nd day onward):
      X_seq = embeddings for all days before day_n: [day_1, ..., day_{n-1}]
      y_vec = EMA multilabel for day_n: [neg, pos]
    Returns:
      X_list: list of np.arrays with shape (T_i, D), T_i varies by sample
      y_arr:  (N, 2)
      subj_arr: (N,)
    """
    X_list = []
    y_list = []
    subj_list = []

    for subj in day_embeddings:
        if subj not in ema_labels_multi:
            continue

        days = sorted(day_embeddings[subj].keys())

        # start at i=7, because i=0 has no history
        for i in range(7, len(days)):
            day_n = days[i]
            prev_day = days[i-1]
            history_days = days[:i]   # all days up to day_{i-1}

            # need EMA label for the target day
            if day_n not in ema_labels_multi[subj]:
                continue
            if prev_day not in ema_labels_multi[subj]:
                continue

            # build sequence of embeddings for all previous days
            seq = np.vstack([day_embeddings[subj][d] for d in history_days])  # (i, D)

            y_vec = ema_labels_multi[subj][day_n]   # shape (2,)

            X_list.append(seq)
            y_list.append(y_vec)
            subj_list.append(subj)

    X_list = np.array(X_list, dtype=object)  # object array since seq lengths differ
    y_arr = np.array(y_list, dtype=np.float32)
    subj_arr = np.array(subj_list)

    return X_list, y_arr, subj_arr


In [57]:
X_cum, y_cum, subj_cum = build_dataset_cumulative_multilabel(day_embeddings, ema_labels_multi)


In [58]:
from torch.utils.data import Dataset, DataLoader

class AffectSequenceDatasetMultiLabelVar(Dataset):
    def __init__(self, X_list, y):
        # X_list: array/list of arrays with shape (T_i, D)
        self.X = [torch.tensor(x, dtype=torch.float32) for x in X_list]
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def collate_varlen_multilabel(batch):
    """
    batch: list of (seq, y) where seq is (T_i, D)
    Returns:
      padded: (B, T_max, D)
      y_batch: (B, 2)
      lengths: (B,) original sequence lengths
    """
    seqs, ys = zip(*batch)
    lengths = [s.shape[0] for s in seqs]
    B = len(seqs)
    D = seqs[0].shape[1]
    T_max = max(lengths)

    padded = torch.zeros(B, T_max, D, dtype=torch.float32)
    for i, s in enumerate(seqs):
        T_i = s.shape[0]
        # left-pad so the most recent day is at the end
        padded[i, T_max - T_i:, :] = s

    y_batch = torch.stack(ys, dim=0)
    lengths = torch.tensor(lengths, dtype=torch.long)

    return padded, y_batch, lengths


In [59]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class GRUAffectMultiLabel(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1, num_labels=2):
        super().__init__()
        self.gru = nn.GRU(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, num_labels)

    def forward(self, x, lengths):
        """
        x: (B, T_max, D)
        lengths: (B,) actual sequence lengths
        """
        # pack padded sequence so GRU ignores left padding
        packed = pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.gru(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)  # (B, T_eff, H)

        # last valid timestep for each sequence
        idx = (lengths - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, out.size(2))
        last_h = out.gather(1, idx).squeeze(1)  # (B, hidden_dim)

        last_h = self.dropout(last_h)
        logits = self.fc(last_h)               # (B, 2)
        return logits


In [60]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix
)
import torch.nn.functional as F
import numpy as np
import torch

def confusion_normalized(cm):
    """
    Row-normalize a confusion matrix.
    cm: (2,2) array
    returns (2,2) float array
    """
    cm = cm.astype(float)
    row_sums = cm.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1.0
    return cm / row_sums


def evaluate_loso_gru_multilabel(
    X, y, subj_ids,
    latent_dim=None,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    """
    X: list/array of length N, each element has shape (T_i, latent_dim)
    y: (N, 2) multilabel targets [neg, pos]
    subj_ids: (N,)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    if latent_dim is None:
        # infer from the first sample
        latent_dim = X[0].shape[1]

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        if not np.any(test_mask):
            continue

        # index arrays for train/test
        train_idx = np.where(train_mask)[0]
        test_idx  = np.where(test_mask)[0]

        # split X into lists of sequences
        X_train_list = [X[i] for i in train_idx]
        X_test_list  = [X[i] for i in test_idx]

        y_train = y[train_mask]
        y_test  = y[test_mask]

        if len(y_test) < 2:
            continue

        #  NORMALIZE (train only)
        # stack all timesteps from training sequences to fit scaler
        concat_train = np.vstack(X_train_list)   # (sum_T_train, D)
        scaler = StandardScaler().fit(concat_train)

        X_train_scaled = [scaler.transform(seq) for seq in X_train_list]
        X_test_scaled  = [scaler.transform(seq) for seq in X_test_list]


        # Datasets & loaders (variable length)
        train_ds = AffectSequenceDatasetMultiLabelVar(X_train_scaled, y_train)
        test_ds  = AffectSequenceDatasetMultiLabelVar(X_test_scaled,  y_test)

        train_loader = DataLoader(
            train_ds, batch_size=batch_size,
            shuffle=True, collate_fn=collate_varlen_multilabel
        )
        test_loader  = DataLoader(
            test_ds, batch_size=batch_size,
            shuffle=False, collate_fn=collate_varlen_multilabel
        )

        # class weights (pos_weight)
        pos_counts = y_train.sum(axis=0)           # (2,)
        neg_counts = len(y_train) - pos_counts

        pos_counts = np.where(pos_counts == 0, 1.0, pos_counts)
        neg_counts = np.where(neg_counts == 0, 1.0, neg_counts)

        raw_pw = neg_counts / pos_counts          # (2,)
        alpha = 0.5
        pos_weight = 1.0 + alpha * (raw_pw - 1.0)

        pos_weight_t = torch.tensor(pos_weight, dtype=torch.float32, device=device)

        #  Model
        model = GRUAffectMultiLabel(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_labels=2
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight_t)

        # TRAIN
        for ep in range(epochs):
            model.train()
            for bx, by, lengths in train_loader:
                bx, by, lengths = bx.to(device), by.to(device), lengths.to(device)
                opt.zero_grad()
                logits = model(bx, lengths)   # (B, 2)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # TRAIN PRED FOR THRESHOLD TUNING
        model.eval()
        train_logits = []
        train_truth = []

        with torch.no_grad():
            for bx, by, lengths in train_loader:
                bx, by, lengths = bx.to(device), by.to(device), lengths.to(device)
                logits = model(bx, lengths)
                train_logits.append(logits.cpu())
                train_truth.append(by)

        train_logits = torch.cat(train_logits, dim=0).cpu()       # (N_train_fold, 2)
        train_truth = torch.cat(train_truth, dim=0).cpu().numpy() # (N_train_fold, 2)
        train_probs = torch.sigmoid(train_logits).cpu().numpy()   # (N_train_fold, 2)

        # Tune thresholds (per label)
        def find_best_threshold(y_true, y_prob, steps=17):
            best_t = 0.5
            best_f1 = 0.0
            for t in np.linspace(0.1, 0.9, steps):
                y_pred = (y_prob >= t).astype(int)
                f1 = f1_score(y_true, y_pred, zero_division=0)
                if f1 > best_f1:
                    best_f1 = f1
                    best_t = t
            return best_t

        thr_neg = find_best_threshold(train_truth[:, 0], train_probs[:, 0])
        thr_pos = find_best_threshold(train_truth[:, 1], train_probs[:, 1])

        # TEST PRED
        test_logits = []
        test_truth = []

        with torch.no_grad():
            for bx, by, lengths in test_loader:
                bx, by, lengths = bx.to(device), by.to(device), lengths.to(device)
                logits = model(bx, lengths)
                test_logits.append(logits.cpu())
                test_truth.append(by)

        test_logits = torch.cat(test_logits, dim=0).cpu()
        test_truth = torch.cat(test_truth, dim=0).cpu().numpy()
        probs = torch.sigmoid(test_logits).cpu().numpy()   # (N_test, 2)

        preds = np.zeros_like(probs, dtype=int)
        preds[:, 0] = (probs[:, 0] >= thr_neg).astype(int)
        preds[:, 1] = (probs[:, 1] >= thr_pos).astype(int)

        #Metrics & confusion matrices
        metrics = {}
        raw_cm_dict = {}
        norm_cm_dict = {}

        for j, name in enumerate(["neg", "pos"]):
            y_true_j = test_truth[:, j]
            y_pred_j = preds[:, j]

            metrics[f"acc_{name}"]  = accuracy_score(y_true_j, y_pred_j)
            metrics[f"prec_{name}"] = precision_score(y_true_j, y_pred_j, zero_division=0)
            metrics[f"rec_{name}"]  = recall_score(y_true_j, y_pred_j, zero_division=0)
            metrics[f"f1_{name}"]   = f1_score(y_true_j, y_pred_j, zero_division=0)

            cm = confusion_matrix(y_true_j, y_pred_j, labels=[0, 1])
            cm_norm = confusion_normalized(cm)

            raw_cm_dict[f"confusion_{name}"] = cm.tolist()
            norm_cm_dict[f"confusion_norm_{name}"] = cm_norm.tolist()

        metrics["acc_macro"]  = 0.5 * (metrics["acc_neg"]  + metrics["acc_pos"])
        metrics["prec_macro"] = 0.5 * (metrics["prec_neg"] + metrics["prec_pos"])
        metrics["rec_macro"]  = 0.5 * (metrics["rec_neg"]  + metrics["rec_pos"])
        metrics["f1_macro"]   = 0.5 * (metrics["f1_neg"]   + metrics["f1_pos"])

        results.append({
            "test_subject": test_subj,
            "n_test": len(test_truth),
            **metrics,
            **raw_cm_dict,
            **norm_cm_dict,
            "pos_weight": pos_weight.tolist(),
            "thr_neg": float(thr_neg),
            "thr_pos": float(thr_pos),
        })

    return pd.DataFrame(results)




In [61]:
set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_cum = evaluate_loso_gru_multilabel(
    X_cum, y_cum, subj_cum,
    latent_dim=X_cum[0].shape[1],   # or omit, it'll infer
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_cum)
print("\nAverage metrics:")
print(loso_cum[["acc_macro", "prec_macro", "rec_macro", "f1_macro"]].mean())


   test_subject  n_test   acc_neg  prec_neg   rec_neg    f1_neg   acc_pos  \
0         cr001      22  0.090909  0.090909  1.000000  0.166667  0.772727   
1         cr002       7  0.285714  0.000000  0.000000  0.000000  0.000000   
2         cr003       7  0.285714  0.285714  1.000000  0.444444  0.285714   
3         cr005      16  0.000000  0.000000  0.000000  0.000000  0.937500   
4         cr006       5  0.400000  0.500000  0.333333  0.400000  0.400000   
5         cr011       9  0.444444  1.000000  0.444444  0.615385  0.333333   
6         cr016       4  0.000000  0.000000  0.000000  0.000000  0.750000   
7         cr031      14  0.928571  0.928571  1.000000  0.962963  0.428571   
8         cr032      12  0.500000  0.200000  0.333333  0.250000  1.000000   
9        crs002      22  1.000000  1.000000  1.000000  1.000000  0.545455   
10       crs007      15  0.266667  0.000000  0.000000  0.000000  0.466667   
11       crs010      18  0.222222  0.176471  1.000000  0.300000  0.500000   

LSTM VERSION OF HR ONLY

In [62]:
class LSTMAffectBinary(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, 1)  # single logit for BCE

    def forward(self, x, lengths):
        """
        x:       (B, T_max, D)
        lengths: (B,)
        """
        packed = pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        # LSTM returns (output, (h_n, c_n))
        packed_out, _ = self.lstm(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)  # (B, T_eff, H)

        # last valid timestep per sample (same as in GRU version)
        idx = (lengths - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, out.size(2))
        last_h = out.gather(1, idx).squeeze(1)  # (B, hidden_dim)

        last_h = self.dropout(last_h)
        logit = self.fc(last_h).squeeze(1)      # (B,)
        return logit


In [63]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
    roc_auc_score,
    confusion_matrix
)

def evaluate_loso_lstm_binary_cumulative_hr(
    X_list, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    """
    X_list: object array/list of length N, each element (T_i, D)
    y:      (N,) 0/1
    subj_ids: (N,)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train_list = X_list[train_mask]
        X_test_list  = X_list[test_mask]
        y_train      = y[train_mask]
        y_test       = y[test_mask]

        # need at least 2 test samples
        if len(y_test) < 2:
            continue

        #  NORMALIZATION (on training only)
        # compute mean/std over all timepoints and dims in training
        all_train_frames = np.vstack(X_train_list)  # (sum T_i, D)
        mean = all_train_frames.mean(axis=0, keepdims=True)  # (1, D)
        std = all_train_frames.std(axis=0, keepdims=True)
        std[std == 0] = 1.0

        def normalize_seq_list(XL):
            out = []
            for arr in XL:
                arr_norm = (arr - mean) / std
                out.append(arr_norm.astype(np.float32))
            return np.array(out, dtype=object)

        X_train_list_norm = normalize_seq_list(X_train_list)
        X_test_list_norm  = normalize_seq_list(X_test_list)


        # datasets/loaders
        train_ds = AffectSequenceDatasetBin_Var(X_train_list_norm, y_train)
        test_ds  = AffectSequenceDatasetBin_Var(X_test_list_norm,  y_test)

        train_loader = DataLoader(
            train_ds, batch_size=batch_size, shuffle=True,
            collate_fn=collate_varlen_bin
        )
        test_loader = DataLoader(
            test_ds, batch_size=batch_size, shuffle=False,
            collate_fn=collate_varlen_bin
        )

        # class imbalance for BCE pos_weight
        class_counts = np.bincount(y_train.astype(int), minlength=2).astype(float)
        if class_counts[1] == 0:
            pos_weight_val = 1.0
        else:
            pos_weight_val = class_counts[0] / class_counts[1]
        pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32, device=device)

        # latent_dim from first sequence
        latent_dim_here = X_train_list_norm[0].shape[1]

        model = LSTMAffectBinary(
              latent_dim=latent_dim_here,
              hidden_dim=hidden_dim,
              num_layers=num_layers
                                ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        # TRAIN
        for ep in range(epochs):
            model.train()
            for bx, by, lengths in train_loader:
                bx = bx.to(device)         # (B, T_max, D)
                by = by.to(device)         # (B,) float
                lengths = lengths.to(device)

                opt.zero_grad()
                logits = model(bx, lengths)  # (B,)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # TRAIN PREDICTIONS FOR THRESHOLD
        model.eval()
        train_logits_all = []
        train_truth_all  = []

        with torch.no_grad():
            for bx, by, lengths in train_loader:
                bx = bx.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths)
                train_logits_all.append(logits.cpu())
                train_truth_all.append(by)

        train_logits_all = torch.cat(train_logits_all, dim=0).numpy()
        train_truth_all  = torch.cat(train_truth_all, dim=0).numpy()
        train_probs_all  = 1.0 / (1.0 + np.exp(-train_logits_all))  # sigmoid

        best_thr, best_f1_train = find_best_threshold(train_truth_all, train_probs_all)

        #  TEST PREDICTIONS
        test_logits_all = []
        test_truth_all  = []

        with torch.no_grad():
            for bx, by, lengths in test_loader:
                bx = bx.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths)
                test_logits_all.append(logits.cpu())
                test_truth_all.append(by)

        test_logits_all = torch.cat(test_logits_all, dim=0).numpy()
        test_truth_all  = torch.cat(test_truth_all, dim=0).numpy()
        test_probs_all  = 1.0 / (1.0 + np.exp(-test_logits_all))

        preds = (test_probs_all >= best_thr).astype(int)
        y_int = test_truth_all.astype(int)

        # metrics
        acc = accuracy_score(y_int, preds)
        f1_macro = f1_score(y_int, preds, average="macro", zero_division=0)
        prec_macro = precision_score(y_int, preds, average="macro", zero_division=0)
        rec_macro = recall_score(y_int, preds, average="macro", zero_division=0)

        rec_per_class = recall_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )
        prec_per_class = precision_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )

        try:
            auc = roc_auc_score(y_int, test_probs_all)
        except ValueError:
            auc = np.nan

        cm = confusion_matrix(y_int, preds, labels=[0, 1])
        cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
        cm_norm = np.nan_to_num(cm_norm)

        results.append({
            "test_subject": test_subj,
            "n_test": len(y_int),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "precision_per_class": prec_per_class.tolist(),
            "confusion_matrix": cm_norm.tolist(),
            "y_test_unique": list(np.unique(y_int)),
            "class_counts_train": class_counts.tolist(),
            "pos_weight": float(pos_weight_val),
            "best_threshold": float(best_thr),
            "train_best_f1": float(best_f1_train),
        })

    return pd.DataFrame(results)


In [64]:
# Build cumulative HR-only dataset
X_hr_cum, y_hr_cum, subj_hr_cum = build_dataset_binary_cumulative_hr(
    day_embeddings, ema_labels_bin
)

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_hr_cum = evaluate_loso_lstm_binary_cumulative_hr(
    X_hr_cum, y_hr_cum, subj_hr_cum,
    latent_dim=X_hr_cum[0].shape[1],   # D from first sequence
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_hr_cum)
print("\nAverage metrics:")
print(loso_hr_cum[["accuracy", "precision_macro", "recall_macro", "f1_macro", "auc"]].mean())


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-4214987100.py:171: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-4214987100.py:171: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-4214987100.py:171: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype(

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      19  0.315789         0.566667      0.617647  0.308123   
1         cr003       7  0.428571         0.458333      0.450000  0.416667   
2         cr005      14  0.500000         0.500000      0.250000  0.333333   
3         cr006       5  0.400000         0.200000      0.500000  0.285714   
4         cr011       4  0.000000         0.000000      0.000000  0.000000   
5         cr016       4  0.500000         0.500000      0.250000  0.333333   
6         cr031       8  0.000000         0.000000      0.000000  0.000000   
7         cr032       9  1.000000         1.000000      1.000000  1.000000   
8        crs002      14  0.571429         0.500000      0.285714  0.363636   
9        crs007      15  1.000000         1.000000      1.000000  1.000000   
10       crs010      17  0.529412         0.600000      0.733333  0.484848   
11       crs011      13  0.000000         0.000000      0.000000

LSTM Version of HR + Prev EMA with cumulative n-1 HR data


In [65]:
class LSTMAffectBinaryWithEMA(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim + 1, 1)  # +1 for prev_ema

    def forward(self, x, lengths, prev_ema):
        packed = pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.lstm(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)

        idx = (lengths - 1).unsqueeze(1).unsqueeze(2).expand(-1, 1, out.size(2))
        last_h = out.gather(1, idx).squeeze(1)   # (B, hidden_dim)

        last_h = self.dropout(last_h)
        prev_ema = prev_ema.unsqueeze(1)         # (B, 1)
        h_cat = torch.cat([last_h, prev_ema], dim=1)  # (B, hidden_dim+1)

        logit = self.fc(h_cat).squeeze(1)        # (B,)
        return logit


In [66]:
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix
)

def evaluate_loso_lstm_binary_with_prev_ema_threshold_varlen(
    X_list, prev_ema, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    """
    X_list: np.array (dtype=object) of length N, each element array(T_i, D)
    prev_ema: (N,)
    y: (N,)  0/1
    subj_ids: (N,)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train_list = X_list[train_mask]
        prev_train   = prev_ema[train_mask]
        y_train      = y[train_mask]

        X_test_list  = X_list[test_mask]
        prev_test    = prev_ema[test_mask]
        y_test       = y[test_mask]

        # need at least 2 test samples
        if len(y_test) < 2:
            continue

        #  datasets / loaders
        train_ds = AffectSequenceDatasetBinWithEMA_Var(X_train_list, prev_train, y_train)
        test_ds  = AffectSequenceDatasetBinWithEMA_Var(X_test_list,  prev_test,  y_test)

        train_loader = DataLoader(
            train_ds, batch_size=batch_size, shuffle=True,
            collate_fn=collate_varlen_bin_with_ema
        )
        test_loader  = DataLoader(
            test_ds, batch_size=batch_size, shuffle=False,
            collate_fn=collate_varlen_bin_with_ema
        )

        #  class imbalance (pos_weight for BCE)
        class_counts = np.bincount(y_train.astype(int), minlength=2).astype(float)
        # class_counts[0] = #neg, class_counts[1] = #pos
        if class_counts[1] == 0:
            pos_weight_val = 1.0
        else:
            pos_weight_val = class_counts[0] / class_counts[1]

        pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32, device=device)

        # model / loss
        # infer latent_dim from first sequence
        if isinstance(X_list[0], np.ndarray):
            latent_dim_here = X_list[0].shape[1]
        else:
            latent_dim_here = latent_dim

        model = LSTMAffectBinaryWithEMA(
              latent_dim=latent_dim_here,
              hidden_dim=hidden_dim,
              num_layers=num_layers
                                ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        #  TRAIN
        for ep in range(epochs):
            model.train()
            for bx, bprev, by, lengths in train_loader:
                bx = bx.to(device)             # (B, T_max, D)
                bprev = bprev.to(device)       # (B,)
                by = by.to(device)             # (B,) float 0/1
                lengths = lengths.to(device)

                opt.zero_grad()
                logits = model(bx, lengths, bprev)  # (B,)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        #  TRAIN PREDICTIONS FOR THRESHOLD TUNING
        model.eval()
        train_logits_all = []
        train_truth_all  = []

        with torch.no_grad():
            for bx, bprev, by, lengths in train_loader:
                bx = bx.to(device)
                bprev = bprev.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths, bprev)  # (B,)
                train_logits_all.append(logits.cpu())
                train_truth_all.append(by)

        train_logits_all = torch.cat(train_logits_all, dim=0).numpy()   # (N_train,)
        train_truth_all  = torch.cat(train_truth_all, dim=0).numpy()    # (N_train,)
        train_probs_all  = 1.0 / (1.0 + np.exp(-train_logits_all))      # sigmoid

        best_thr, best_f1_train = find_best_threshold(train_truth_all, train_probs_all)

        #  TEST PREDICTIONS
        test_logits_all = []
        test_truth_all  = []

        with torch.no_grad():
            for bx, bprev, by, lengths in test_loader:
                bx = bx.to(device)
                bprev = bprev.to(device)
                lengths = lengths.to(device)
                logits = model(bx, lengths, bprev)
                test_logits_all.append(logits.cpu())
                test_truth_all.append(by)

        test_logits_all = torch.cat(test_logits_all, dim=0).numpy()  # (N_test,)
        test_truth_all  = torch.cat(test_truth_all, dim=0).numpy()   # (N_test,)
        test_probs_all  = 1.0 / (1.0 + np.exp(-test_logits_all))

        preds = (test_probs_all >= best_thr).astype(int)
        y_int = test_truth_all.astype(int)

        # metrics
        acc = accuracy_score(y_int, preds)
        f1_macro = f1_score(y_int, preds, average="macro", zero_division=0)
        prec_macro = precision_score(y_int, preds, average="macro", zero_division=0)
        rec_macro = recall_score(y_int, preds, average="macro", zero_division=0)

        rec_per_class = recall_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )
        prec_per_class = precision_score(
            y_int, preds,
            average=None,
            labels=[0, 1],
            zero_division=0
        )

        try:
            auc = roc_auc_score(y_int, test_probs_all)
        except ValueError:
            auc = np.nan

        cm = confusion_matrix(y_int, preds, labels=[0, 1])
        cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
        cm_norm = np.nan_to_num(cm_norm)

        results.append({
            "test_subject": test_subj,
            "n_test": len(y_int),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "precision_per_class": prec_per_class.tolist(),
            "confusion_matrix": cm_norm.tolist(),
            "y_test_unique": list(np.unique(y_int)),
            "class_counts_train": class_counts.tolist(),
            "pos_weight": float(pos_weight_val),
            "best_threshold": float(best_thr),
            "train_best_f1": float(best_f1_train),
        })

    return pd.DataFrame(results)




In [67]:
X_bin, prev_bin, y_bin, subj_bin = build_dataset_binary_cumulative_with_prev_ema(
    day_embeddings, ema_labels_bin
)

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_bin_cum = evaluate_loso_lstm_binary_with_prev_ema_threshold_varlen(
    X_bin, prev_bin, y_bin, subj_bin,
    latent_dim=X_bin[0].shape[1],   # D from first sequence
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_bin_cum)

print("\nAverage metrics:")
print(loso_bin_cum[["accuracy", "precision_macro", "recall_macro", "f1_macro", "auc"]].mean())



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-3061278158.py:162: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-3061278158.py:162: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-3061278158.py:162: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype(

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      19  0.315789         0.566667      0.617647  0.308123   
1         cr003       7  0.571429         0.541667      0.550000  0.533333   
2         cr005      14  0.357143         0.500000      0.178571  0.263158   
3         cr006       5  0.600000         0.750000      0.666667  0.583333   
4         cr011       4  0.000000         0.000000      0.000000  0.000000   
5         cr016       4  0.500000         0.500000      0.250000  0.333333   
6         cr031       8  0.875000         0.500000      0.437500  0.466667   
7         cr032       9  0.444444         0.500000      0.222222  0.307692   
8        crs002      14  1.000000         1.000000      1.000000  1.000000   
9        crs007      15  0.266667         0.500000      0.133333  0.210526   
10       crs010      17  0.529412         0.600000      0.733333  0.484848   
11       crs011      13  0.307692         0.500000      0.153846

LSTM with Fixed H- HR only

In [68]:
class LSTMAffectClassifier(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        """
        x: (batch, H, latent_dim)
        """
        out, _ = self.lstm(x)           # (batch, H, hidden_dim)
        h_last = out[:, -1, :]          # last timestep
        logits = self.fc(h_last)        # (batch, num_classes)
        return logits


In [69]:
def evaluate_loso_lstm_binary(
    X, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train, y_train = X[train_mask], y[train_mask]
        X_test,  y_test  = X[test_mask],  y[test_mask]

        # we skip if test subject has < 2 samples OR only 1 class (AUC would be degenerate)
        if len(y_test) < 2: #or len(np.unique(y_test)) < 2
            continue

        N_train,H, D = X_train.shape
        N_test = X_test.shape[0]

        X_train_flat = X_train.reshape(N_train, -1)
        X_test_flat = X_test.reshape(N_test, -1)

        scalar= StandardScaler()
        X_train_scaled = scalar.fit_transform(X_train_flat)
        X_test_scaled = scalar.transform(X_test_flat)

        #reshape back to (N, H, latent_dim)
        X_train = X_train_scaled.reshape(N_train, H, D)
        X_test = X_test_scaled.reshape(N_test, H, D)

        train_ds = AffectSequenceDatasetBin(X_train, y_train)
        test_ds  = AffectSequenceDatasetBin(X_test,  y_test)

        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

        #  class weights (to handle imbalance)
        class_counts = np.bincount(y_train, minlength=2).astype(float)
        class_counts[class_counts == 0.0] = 1.0
        class_weights = 1.0 / class_counts
        class_weights = class_weights * (2 / class_weights.sum())
        weight_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)

        model = LSTMAffectClassifier(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_classes=2
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.CrossEntropyLoss(weight=weight_tensor)

        #  Train
        for ep in range(epochs):
            model.train()
            for bx, by in train_loader:
                bx, by = bx.to(device), by.to(device)
                opt.zero_grad()
                logits = model(bx)              # (batch, 2)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # Evaluate
        model.eval()
        all_logits = []
        all_truth = []

        with torch.no_grad():
            for bx, by in test_loader:
                bx = bx.to(device)
                logits = model(bx)
                all_logits.append(logits.cpu())
                all_truth.append(by)

        all_logits = torch.cat(all_logits, dim=0)        # (N_test, 2)
        all_truth = torch.cat(all_truth, dim=0).numpy()  # (N_test,)
        probs = F.softmax(all_logits, dim=1).numpy()     # (N_test, 2)
        preds = probs.argmax(axis=1)                     # (N_test,)
        pos_probs = probs[:, 1]                          # for AUC

        # Metrics
        acc = accuracy_score(all_truth, preds)
        f1_macro = f1_score(all_truth, preds, average="macro", zero_division=0)
        prec_macro = precision_score(all_truth, preds, average="macro", zero_division=0)
        rec_macro = recall_score(all_truth, preds, average="macro", zero_division=0)

        # per-class recall [0, 1]
        label_list = [0, 1]
        rec_per_class = recall_score(
            all_truth, preds,
            average=None,
            labels=label_list,
            zero_division=0
        )

        try:
            auc = roc_auc_score(all_truth, pos_probs)
        except ValueError:
            auc = np.nan

        # Confusion matrix
        cm = confusion_matrix(all_truth, preds, labels=label_list)

        results.append({
            "test_subject": test_subj,
            "n_test": len(all_truth),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "confusion_matrix": cm.tolist(),
            "y_test_unique": list(np.unique(all_truth)),
            "class_counts_train": class_counts.tolist(),
            "class_weights": class_weights.tolist(),
        })

    return pd.DataFrame(results)

In [70]:
set_seed(42)
latent_dim = X_mc.shape[2]   # 64
device = "cuda" if torch.cuda.is_available() else "cpu"

loso_lstm = evaluate_loso_lstm_binary(
    X_mc, y_mc, subj_mc,
    latent_dim=latent_dim,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_lstm)
print("\nAverage metrics across subjects:")
print(loso_lstm[["accuracy", "precision_macro", "recall_macro", "f1_macro", "auc"]].mean())


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      21  0.476190         0.416667      0.263158  0.322581   
1         cr003       7  0.285714         0.142857      0.500000  0.222222   
2         cr004       2  0.500000         0.250000      0.500000  0.333333   
3         cr005      15  0.866667         0.500000      0.433333  0.464286   
4         cr006      14  0.285714         0.250000      0.200000  0.222222   
5         cr011       7  0.285714         0.500000      0.142857  0.222222   
6         cr016      10  1.000000         1.000000      1.000000  1.000000   
7         cr031      19  0.789474         0.604167      0.604167  0.604167   
8         cr032      11  0.818182         0.500000      0.409091  0.450000   
9        crs002      17  0.529412         0.500000      0.264706  0.346154   
10       crs007      15  0.066667         0.500000      0.033333  0.062500   
11       crs010      20  0.300000         0.588235      0.588235

LSTM With Fixed H- HR+Prev EMA

In [71]:

class LSTMAffectClassifierWithEMA(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=64, num_layers=1, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.dropout = nn.Dropout(0.3)
        # +1 to concatenate prev_ema as extra feature
        self.fc = nn.Linear(hidden_dim + 1, num_classes)

    def forward(self, x, prev_ema):
        """
        x:        (batch, H, latent_dim)
        prev_ema: (batch,)  scalar 0/1 for previous day's affect
        """
        out, (h_n, c_n) = self.lstm(x)        # out: (B, H, hidden_dim), h_n: (num_layers, B, hidden_dim)
        h_last = out[:, -1, :]                # last time step (B, hidden_dim)
        h_last = self.dropout(h_last)

        prev_ema = prev_ema.unsqueeze(1)      # (B, 1)
        h_cat = torch.cat([h_last, prev_ema], dim=1)  # (B, hidden_dim + 1)

        logits = self.fc(h_cat)               # (B, num_classes)
        return logits


In [72]:
EMBEDDING_DIM = 64

def build_dataset_binary_fixedH_with_prev_ema(day_embeddings, ema_labels_bin, H=7):
    """
    For each subject:
      - Input X: (H, D) embeddings for days [n-H, ..., n-1]
      - prev_ema: EMA_bin at day (n-1)
      - Target y: EMA_bin at day n

    Returns:
      X:        (N, H, D)
      prev_ema: (N,)
      y:        (N,)
      subj_ids: (N,)
    """
    import numpy as np

    X_list, prev_ema_list, y_list, subj_list = [], [], [], []

    for subj in day_embeddings:
        if subj not in ema_labels_bin:
            continue

        days = sorted(day_embeddings[subj].keys())

        for i in range(H, len(days)):
            day_n = days[i]          # prediction day
            prev_days = days[i-H:i]  # H previous days
            day_prev = days[i-1]     # immediate previous day

            # need labels for day_n and day_prev
            if day_n not in ema_labels_bin[subj]:
                continue
            if day_prev not in ema_labels_bin[subj]:
                continue

            try:
                seq = np.vstack([day_embeddings[subj][d] for d in prev_days])  # (H, D)
            except KeyError:
                continue

            X_list.append(seq)
            y_list.append(ema_labels_bin[subj][day_n])
            prev_ema_list.append(ema_labels_bin[subj][day_prev])
            subj_list.append(subj)

    X = np.array(X_list, dtype=np.float32)
    prev_ema = np.array(prev_ema_list, dtype=np.float32)
    y = np.array(y_list, dtype=int)
    subj_ids = np.array(subj_list)

    return X, prev_ema, y, subj_ids


In [73]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix
)
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn

def find_best_threshold(y_true, y_prob, steps=17):
    """
    y_true: (N,) binary 0/1
    y_prob: (N,) predicted prob of class 1
    """
    best_t = 0.5
    best_f1 = -1.0
    for t in np.linspace(0.1, 0.9, steps):
        y_pred = (y_prob >= t).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
    return best_t, best_f1


def evaluate_loso_lstm_binary_with_prev_ema_fixedH(
    X, prev_ema, y, subj_ids,
    latent_dim=64,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=None
):
    """
    X:        (N, H, D)  HR embeddings
    prev_ema: (N,)       previous day's EMA (0/1)
    y:        (N,)       target EMA (0/1)
    subj_ids: (N,)       subject ids
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    unique_subjs = np.unique(subj_ids)
    results = []

    for test_subj in unique_subjs:
        test_mask = (subj_ids == test_subj)
        train_mask = ~test_mask

        X_train, prev_train, y_train = X[train_mask], prev_ema[train_mask], y[train_mask]
        X_test,  prev_test,  y_test  = X[test_mask],  prev_ema[test_mask],  y[test_mask]

        # skip tiny test sets
        if len(y_test) < 2:
            continue

        # Normalize features (train only)
        N_train, H, D = X_train.shape
        N_test = X_test.shape[0]

        X_train_flat = X_train.reshape(N_train, -1)
        X_test_flat  = X_test.reshape(N_test,  -1)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_flat)
        X_test_scaled  = scaler.transform(X_test_flat)

        X_train = X_train_scaled.reshape(N_train, H, D)
        X_test  = X_test_scaled.reshape(N_test,  H, D)

        train_ds = AffectSequenceDatasetBinWithEMA(X_train, prev_train, y_train)
        test_ds  = AffectSequenceDatasetBinWithEMA(X_test,  prev_test,  y_test)

        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

        #  class weights (handle imbalance)
        class_counts = np.bincount(y_train, minlength=2).astype(float)
        class_counts[class_counts == 0.0] = 1.0
        class_weights = 1.0 / class_counts
        class_weights = class_weights * (2 / class_weights.sum())
        weight_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)

        # Model
        model = LSTMAffectClassifierWithEMA(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_classes=2
        ).to(device)

        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.CrossEntropyLoss(weight=weight_tensor)

        # Train
        for ep in range(epochs):
            model.train()
            for bx, bprev, by in train_loader:
                bx, bprev, by = bx.to(device), bprev.to(device), by.to(device)
                opt.zero_grad()
                logits = model(bx, bprev)        # (B, 2)
                loss = loss_fn(logits, by)
                loss.backward()
                opt.step()

        # Train predictions for threshold tuning
        model.eval()
        train_logits_all = []
        train_truth_all  = []

        with torch.no_grad():
            for bx, bprev, by in train_loader:
                bx, bprev = bx.to(device), bprev.to(device)
                logits = model(bx, bprev)
                train_logits_all.append(logits.cpu())
                train_truth_all.append(by)

        train_logits_all = torch.cat(train_logits_all, dim=0).cpu()        # (N_train_fold, 2)
        train_truth_all  = torch.cat(train_truth_all, dim=0).cpu().numpy() # (N_train_fold,)
        train_probs_all  = F.softmax(train_logits_all, dim=1).cpu().numpy()[:, 1]

        best_thr, best_f1_train = find_best_threshold(train_truth_all, train_probs_all)

        # Test predictions
        test_logits_all = []
        test_truth_all  = []

        with torch.no_grad():
            for bx, bprev, by in test_loader:
                bx, bprev = bx.to(device), bprev.to(device)
                logits = model(bx, bprev)
                test_logits_all.append(logits.cpu())
                test_truth_all.append(by)

        test_logits_all = torch.cat(test_logits_all, dim=0).cpu()
        test_truth_all  = torch.cat(test_truth_all, dim=0).cpu().numpy()
        test_probs_all  = F.softmax(test_logits_all, dim=1).cpu().numpy()[:, 1]

        preds = (test_probs_all >= best_thr).astype(int)

        # Metrics
        acc = accuracy_score(test_truth_all, preds)
        f1_macro = f1_score(test_truth_all, preds, average="macro", zero_division=0)
        prec_macro = precision_score(test_truth_all, preds, average="macro", zero_division=0)
        rec_macro = recall_score(test_truth_all, preds, average="macro", zero_division=0)

        label_list = [0, 1]
        rec_per_class = recall_score(
            test_truth_all, preds,
            average=None,
            labels=label_list,
            zero_division=0
        )
        prec_per_class = precision_score(
            test_truth_all, preds,
            average=None,
            labels=label_list,
            zero_division=0
        )

        try:
            auc = roc_auc_score(test_truth_all, test_probs_all)
        except ValueError:
            auc = np.nan

        cm = confusion_matrix(test_truth_all, preds, labels=label_list)
        cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
        cm_norm = np.nan_to_num(cm_norm)

        results.append({
            "test_subject": test_subj,
            "n_test": len(test_truth_all),
            "accuracy": acc,
            "precision_macro": prec_macro,
            "recall_macro": rec_macro,
            "f1_macro": f1_macro,
            "auc": auc,
            "recall_per_class": rec_per_class.tolist(),
            "precision_per_class": prec_per_class.tolist(),
            "confusion_matrix": cm_norm.tolist(),
            "y_test_unique": list(np.unique(test_truth_all)),
            "class_counts_train": class_counts.tolist(),
            "class_weights": class_weights.tolist(),
            "best_threshold": float(best_thr),
            "train_best_f1": float(best_f1_train),
        })

    return pd.DataFrame(results)


In [74]:
# Build fixed-H dataset (e.g., H = 7 days history)
H = 7
X_H, prev_H, y_H, subj_H = build_dataset_binary_fixedH_with_prev_ema(
    day_embeddings, ema_labels_bin, H=H
)

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

latent_dim = X_H.shape[2]  # 32 or 64 depending on autoencoder

loso_lstm_H = evaluate_loso_lstm_binary_with_prev_ema_fixedH(
    X_H, prev_H, y_H, subj_H,
    latent_dim=latent_dim,
    hidden_dim=64,
    num_layers=1,
    epochs=30,
    batch_size=16,
    device=device
)

print(loso_lstm_H)
print("\nAverage metrics:")
print(loso_lstm_H[["accuracy","precision_macro","recall_macro","f1_macro","auc"]].mean())


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-2411081043.py:170: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-2411081043.py:170: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/tmp/ipython-input-2411081043.py:170: RuntimeWarning: invalid value encountered in divide
  cm_norm = cm.astype(

   test_subject  n_test  accuracy  precision_macro  recall_macro  f1_macro  \
0         cr001      19  0.526316         0.416667      0.294118  0.344828   
1         cr003       7  0.285714         0.142857      0.500000  0.222222   
2         cr005      14  0.928571         0.500000      0.464286  0.481481   
3         cr006       5  0.200000         0.125000      0.250000  0.166667   
4         cr011       4  1.000000         1.000000      1.000000  1.000000   
5         cr016       4  1.000000         1.000000      1.000000  1.000000   
6         cr031       8  0.625000         0.500000      0.312500  0.384615   
7         cr032       9  1.000000         1.000000      1.000000  1.000000   
8        crs002      14  0.428571         0.500000      0.214286  0.300000   
9        crs007      15  0.200000         0.500000      0.100000  0.166667   
10       crs010      17  0.235294         0.566667      0.566667  0.235294   
11       crs011      13  0.307692         0.500000      0.153846